<a href="https://colab.research.google.com/github/dariabokareva/datascience/blob/master/neural-networks/nlp/kaggle/dl_nlp_toxic_comment_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Классификация [токсичных комментариев](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)

In [1]:
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling1D, Dropout, LSTM, Bidirectional, SpatialDropout1D
from tensorflow.keras import utils
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import utils
from google.colab import files
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
# Максимальное количество слов 
num_words = 10000
# Максимальная длина комментария
max_comment_len = 50

## Настраиваем Kaggle API

Загружаем [Kaggle API Tocken для аутентификации на Kaggle](https://www.kaggle.com/docs/api). 
My Account -> API -> Create New API Token



In [3]:
api_tocken = files.upload()

Saving kaggle.json to kaggle.json


In [4]:
!mkdir /root/.kaggle
!mv kaggle.json /root/.kaggle

## Загрузка набора данных

In [7]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

In [ ]:
!unzip train.csv.zip

In [ ]:
!unzip test.csv.zip
!unzip test_labels.csv.zip

## Просматриваем данные

In [ ]:
!ls

In [ ]:
!head -20 train.csv

## Загружаем данные в память

Читаем данные из файла

In [ ]:
train = pd.read_csv('train.csv')

In [ ]:
train

Выделяем данные для обучения

In [ ]:
comments = train['comment_text']

In [ ]:
comments[:5]

Выделяем правильные ответы

In [ ]:
y_train = train[['toxic', 'severe_toxic', 'obscene',	'threat',	'insult',	'identity_hate']]

In [ ]:
y_train

## Токенизация текста

In [ ]:
comments[:5]

Создаем токенизатор Keras

In [ ]:
tokenizer = Tokenizer(num_words=num_words)

Обучаем токенизатор на новостях

In [ ]:
tokenizer.fit_on_texts(comments)

Просматриваем словарь токенизатора

In [ ]:
tokenizer.word_index

Преобразуем комментарии в числовое представление

In [ ]:
sequences = tokenizer.texts_to_sequences(comments)

Просматриваем новости в числовом представлении

In [ ]:
index = 1
print(comments[index])
print(sequences[index])

In [ ]:
tokenizer.word_index['he']

Ограничиваем длину отзывов

In [ ]:
x_train = pad_sequences(sequences, maxlen=max_comment_len)

In [ ]:
x_train[:5]

## Cеть LSTM

In [ ]:
model_lstm = Sequential()
model_lstm.add(Embedding(num_words, 128, input_length=max_comment_len))
model_lstm.add(SpatialDropout1D(0.5))
model_lstm.add(LSTM(40, return_sequences=True))
model_lstm.add(LSTM(40))
model_lstm.add(Dense(6, activation='sigmoid'))

In [ ]:
model_lstm.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy', 'AUC'])

In [ ]:
model_lstm.summary()

Создаем callback для сохранения нейронной сети на каждой эпохе, если качество работы на проверочном наборе данных улучшилось. Сеть сохраняется в файл `best_model.h5`

In [ ]:
model_lstm_save_path = 'best_model_lstm.h5'
checkpoint_callback_lstm = ModelCheckpoint(model_lstm_save_path, 
                                      monitor='val_accuracy',
                                      save_best_only=True,
                                      verbose=1)

In [ ]:
history_lstm = model_lstm.fit(x_train, 
                              y_train, 
                              epochs=5,
                              batch_size=512,
                              validation_split=0.2,
                              callbacks=[checkpoint_callback_lstm])

In [ ]:
plt.plot(history_lstm.history['accuracy'], 
         label='Доля верных ответов на обучающем наборе')
plt.plot(history_lstm.history['val_accuracy'], 
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

## Загружаем набор данных для тестирования

In [ ]:
test_comments = pd.read_csv('test.csv')

In [ ]:
test_comments[:10]

In [ ]:
test_labels = pd.read_csv('test_labels.csv')

In [ ]:
test_labels[:10]

In [ ]:
test_full = pd.merge(test_comments, test_labels, on='id')

In [ ]:
test_full

In [ ]:
test = test_full[test_full['toxic']!=-1]

In [ ]:
test

Преобразуем комментарии в числовое представление

In [ ]:
test_sequences = tokenizer.texts_to_sequences(test['comment_text'])

In [ ]:
x_test = pad_sequences(test_sequences, maxlen=max_comment_len)

In [ ]:
x_test[:5]

Правильные ответы

In [ ]:
y_test = test[['toxic',	'severe_toxic',	'obscene', 'threat',	'insult',	'identity_hate']]

In [ ]:
y_test[:10]

## Оцениваем качество работы сети на тестовом наборе данных

In [ ]:
model_lstm.load_weights(model_lstm_save_path)

In [ ]:
model_lstm.evaluate(x_test, y_test, verbose=1)

## Пробуем классифицировать комментарии

In [ ]:
comment = "X-BOX 360 SUKCS BIG BUMM AND LIKES IT UP THE ASS"

In [ ]:
sequence = tokenizer.texts_to_sequences([comment])

In [ ]:
sequence

In [ ]:
data = pad_sequences(sequence, maxlen=max_comment_len)

In [ ]:
data

In [ ]:
result = model_lstm.predict(data)

In [ ]:
result